In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **1. Importing libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC


In [ ]:
import nltk # 문장 토크나이저

# 영어 불용어 - 불용어 모아 놓은 리스트 다운로드해 제거
nltk.download('all')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize # 토큰화
from nltk.stem.porter import PorterStemmer # 어근 동일화 <-> 이거 말고도 "Lancaster Stemmer"

# 표제어 추출
from nltk.stem import WordNetLemmatizer

# 정규표현 처리
import re

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/testnew1')
train = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/trainnew1')

In [ ]:
train_df = train.copy()

In [ ]:
train_df.head()

,Unnamed: 0,ID,first_party,second_party,fact,issued_area,sen_len,word_len,winning_percent,first_party_winner,승률범주형분류,fact_processing
0,0,TRAIN_0000,PERSON,PERSON,"On June 27, 1962, <p1> a candidate for public ...",others,7,201,0.5,1,1,june <p1> candid public offic make televis spe...
1,1,TRAIN_0001,PERSON,PERSON,Ramon Nelson was riding his bike when he suffe...,criminal,7,219,0.5,0,1,ramon nelson rid bike suffer lethal blow back ...
2,2,TRAIN_0002,PERSON,GPE,An Alabama state court convicted <p1> of murde...,criminal,8,191,0.5,1,1,alabama state court convict <p1> murder senten...
3,3,TRAIN_0003,others,others,Victor <p1> was convicted in state court on ev...,others,3,59,0.5,0,1,victor <p1> convict state court evid illeg obt...
4,4,TRAIN_0004,PERSON,GPE,"On April 24, 1953 in Selma, <p2> an intruder b...",others,9,200,0.5,1,1,april selma <p2> intrud break apart daughter c...


In [ ]:
test_df = test.copy()

train, test split

In [ ]:
# import random

# from sklearn.model_selection import train_test_split
# train_df, val_df = train_test_split(train, test_size=0.2, random_state=40)
# train_df = train_df.reset_index(drop=True)
# val_df = val_df.reset_index(drop=True)

In [ ]:
# train_df['winner']=0
# train_df['winner_frequency']=0
# train_df['win_percentage']=0

# for i in range(len(train_df)):
#   if train_df['first_party_winner'][i]==1:
#     train_df['winner'][i] = train_df['first_party'][i]
#   else:
#     train_df['winner'][i] = train_df['second_party'][i]

# train_df['first_party_win_percent'] = 0
# train_df['first_party_frequency'] = 0

# for i in range(len(train_df)):
#   train_df['first_party_frequency'][i] = len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]])

#   if (len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]])) > 1:
#     train_df['first_party_win_percent'][i]= len(train_df.loc[train_df['winner'] == train_df['first_party'][i]]) /(len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]]))
#   else:
#     train_df['first_party_win_percent'][i] = 0.5

# train_df=train_df.drop(['winner',	'winner_frequency','first_party_frequency','win_percentage'],axis=1)

In [ ]:
# val_df_first_party = val_df['first_party']
# train_df_first_party = train_df[['first_party','first_party_win_percent']]
# train_df_first_party=train_df_first_party.drop_duplicates()
# test_first_party_병합 = pd.merge(val_df_first_party, train_df_first_party, how='left')
# test_first_party_병합 = test_first_party_병합.fillna(0.5)
# val_df['first_party_win_percent'] = test_first_party_병합['first_party_win_percent']

In [ ]:
def categorize_probability(x):
    if x == 0:
        return 0
    elif 0 < x <= 0.33:
        return 1
    elif 0.33 < x < 0.5:
        return 2
    elif x == 0.5:
        return 3
    elif 0.5 < x < 0.57:
        return 2
    elif 0.57 <= x < 0.75:
        return 4
    elif 0.75 <= x < 1:
        return 5
    elif x == 1:
        return 6
    else:
        return -1

In [ ]:
# def categorize_probability(x):
#     if x < 0.2:
#       return 'lose'
#     elif x >= 0.8:
#       return 'win'
#     else:
#       return "미정"

In [ ]:
train_df['승률범주형분류'] = 0

In [ ]:
for i in range(len(train_df)):
  train_df['승률범주형분류'][i] = categorize_probability(train_df['winning_percent'][i])

In [ ]:
test_df['승률범주형분류'] = 0

for i in range(len(test_df)):
  test_df['승률범주형분류'][i] = categorize_probability(test_df['winning_percent'][i])

In [ ]:
train_df['승률범주형분류'].value_counts()

3    2023
4     231
6      92
2      71
1      26
5      19
0      16
Name: 승률범주형분류, dtype: int64

In [ ]:
from nltk.corpus import names #corpus=말뭉치,이름 관련 부분 다루기 위한 객체


In [ ]:
# 영어 데이터 전처리 함수
stops = set(stopwords.words('english'))
ps = nltk.stem.porter.PorterStemmer()
all_names=set(names.words())
lem = nltk.stem.wordnet.WordNetLemmatizer()

def cleaning(str):
    replaceAll = str

    # 특수문자 및 기호 등 필요없는 문자 제거
    words = replaceAll.split()
    only_english = ''
    for word in words:
      if word in ['<p1>','<p2>']:
        only_english = only_english + word + ' '
      else:
        only_english += re.sub(r"[^a-zA-Z]", ' ', word)
        only_english += ' '  # 띄어쓰기 추가
    only_english = only_english.strip()

    # 대소문자 모두 소문자로 통일
    no_capitals = only_english.lower().split()

    # 이름, 불용어(분석에 필요없는 토큰) 제거
    all_names=set(names.words())
    no_stops = [word for word in no_capitals if not word in all_names|stops]

    # 표제어 : 단어의 원형 형태를 나타내며, 명사의 경우 복수형이나 동사의 경우 시제 등을 고려하여 변환
    lem_text = [lem.lemmatize(word, pos='v') for word in no_stops]

    # 어근 추츨 : 단어의 형태를 보존하는 특징이 있지만 추출된 어근이 실제로는 사전에 존재하지 않은 단어일 수 있음
    stemmer_words = [ps.stem(word) for word in lem_text]

    # back to string from list
    text = " ".join(stemmer_words)

    return text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features = 400)

def get_vector(vectorizer, df, train_mode):
    if train_mode:
        X_facts = vectorizer.fit_transform(df['fact_processing'])
    else:
        X_facts = vectorizer.transform(df['fact_processing'])

    X = np.concatenate([X_facts.todense()], axis=1)
    return X

In [ ]:
# 데이터 클리닝
train_df["fact_processing"] = train_df["fact"].apply(cleaning)

In [ ]:
test_df["fact_processing"] = test_df["fact"].apply(cleaning)

In [ ]:
train_df.to_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/trainnew1')

In [ ]:
test_df.to_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/testnew1')

In [ ]:
train_df['first_party'].astype("category")
train_df['second_party'].astype("category")
train_df['issued_area'].astype("category")

0         others
1       criminal
2       criminal
3         others
4         others
          ...   
2473      others
2474      others
2475    criminal
2476      others
2477      others
Name: issued_area, Length: 2478, dtype: category
Categories (3, object): ['civil', 'criminal', 'others']

In [ ]:
test_df['first_party'].astype("category")
test_df['second_party'].astype("category")
test_df['issued_area'].astype("category")

0         others
1       criminal
2       criminal
3         others
4         others
          ...   
1235    criminal
1236    criminal
1237      others
1238      others
1239      others
Name: issued_area, Length: 1240, dtype: category
Categories (3, object): ['civil', 'criminal', 'others']

In [ ]:
# # 특정 범주형 변수(categorical variable)을 더미변수 생성
# train_cat = pd.get_dummies(data = train_df[["first_party","second_party","issued_area"]])
# test_cat = pd.get_dummies(data = test_df[["first_party","second_party","issued_area"]])
# 특정 범주형 변수(categorical variable)을 더미변수 생성
train_cat = pd.get_dummies(data = train_df[["first_party","second_party","issued_area"]])
test_cat = pd.get_dummies(data = test_df[["first_party","second_party","issued_area"]])

In [ ]:
# 데이터 벡터화
train_fact = get_vector(vectorizer, train_df, True)
train_fact = np.asarray(train_fact)
train_fact = pd.DataFrame(data=train_fact)

test_fact = get_vector(vectorizer, test_df, False)
test_fact = np.asarray(test_fact)
test_fact = pd.DataFrame(data=test_fact)

In [ ]:
train_num = train_df['승률범주형분류']
train_target = train_df[['first_party_winner']]

test_num = test_df['승률범주형분류']

In [ ]:
train_fin = pd.concat([train_fact,train_cat,train_num,train_target],axis=1,join='inner')
test_fin = pd.concat([test_fact, test_cat, test_num],axis=1,join='inner')

In [ ]:
X_train = train_fin.drop(columns=['first_party_winner'])
y_train = train_fin['first_party_winner']

X_test = test_fin

In [ ]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

지금부터 교차검증 수행

In [ ]:
from sklearn.model_selection import cross_val_score


In [ ]:
logreg = LogisticRegression()
scores = cross_val_score(logreg, X_train, y_train, cv=5, scoring='accuracy')

print('Cross-Validation Scores:', scores)
print('Average Score:', scores.mean())

Cross-Validation Scores: [0.67540323 0.68346774 0.66330645 0.65454545 0.65858586]
Average Score: 0.6670617464972304


랜덤포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
rdf = RandomForestClassifier()
scores = cross_val_score(rdf, X_train, y_train, cv=5, scoring='accuracy')

print('Cross-Validation Scores:', scores)
print('Average Score:', scores.mean())

Cross-Validation Scores: [0.66532258 0.65524194 0.67741935 0.65858586 0.66666667]
Average Score: 0.6646472792440534


KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn = knn=KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')

print('Cross-Validation Scores:', scores)
print('Average Score:', scores.mean())

Cross-Validation Scores: [0.64112903 0.56653226 0.62701613 0.60606061 0.61010101]
Average Score: 0.6101678071032909


CATBOOST

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
counts = list(y_train.value_counts())
class_weight = [counts[1]/sum(counts), counts[0]/sum(counts)]
print("weight :", class_weight)

weight : [0.33454398708635996, 0.66545601291364]


In [ ]:
cbc = CatBoostClassifier(random_seed=42,class_weights=class_weight, verbose=0)
scores = cross_val_score(cbc, X_train, y_train, cv=5, scoring='accuracy')

print('Cross-Validation Scores:', scores)
print('Average Score:', scores.mean())

새로운 모델 등장

In [ ]:
!pip install --upgrade tensorflow


In [ ]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 43.7 MB/s eta 0:00:00


In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
from sklearn.utils import resample


In [ ]:
subset_0 = train_df[train_df["first_party_winner"] == 0]
subset_1 = train_df[train_df["first_party_winner"] == 1]

subset_1_downsampled = resample(subset_1,
                                replace=False,
                                n_samples=800,
                                random_state=42)

train_df = pd.concat([subset_0, subset_1_downsampled])

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
numeric_input = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name="numeric_input")
categorical_input = tf.keras.layers.Input(shape=(9,), dtype=tf.int32, name="categorical_input")

preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

processed_numeric = tf.keras.layers.Dense(32, activation="relu")(numeric_input)
processed_categorical = tf.keras.layers.Embedding(9, 32)(categorical_input)
processed_categorical = tf.keras.layers.Flatten()(processed_categorical)

combined_outputs = tf.keras.layers.Concatenate()([outputs["pooled_output"], processed_numeric, processed_categorical])

l = tf.keras.layers.Dropout(0.1, name="dropout")(combined_outputs)
l = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(l)

model = tf.keras.Model(inputs=[text_input, numeric_input, categorical_input], outputs=[l])
model.summary()

METRICS = [
    tf.keras.metrics.BinaryAccuracy(name="accuracy")
]

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=METRICS
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 categorical_input (InputLayer)  [(None, 9)]         0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                  

In [ ]:
X_train.columns

Index([                     0,                      1,                      2,
                            3,                      4,                      5,
                            6,                      7,                      8,
                            9,
       ...
         'first_party_PERSON',   'first_party_others',     'second_party_GPE',
           'second_party_ORG',  'second_party_PERSON',  'second_party_others',
          'issued_area_civil', 'issued_area_criminal',   'issued_area_others',
                    '승률범주형분류'],
      dtype='object', length=412)

In [ ]:
y_train

1       0
3       0
4       1
5       1
7       1
       ..
1617    0
1620    0
1621    0
1624    1
1627    1
Name: first_party_winner, Length: 1068, dtype: int64

In [ ]:
X_train['first_party_winner'] = train_df['first_party_winner']


In [ ]:
X_train.shape

(1068, 413)

In [ ]:
train_df["fact_processing"]

1       ramon nelson rid bike suffer lethal blow back ...
3       victor <p1> convict state court evid illeg obt...
14      name plaintiff <p1> thole other bring class ac...
16      revis texa educ law allow state withhold local...
21      maher kara join citigroup healthcar invest ban...
                              ...                        
951     plaintiff case compris <p1> labor organ sue <p...
1360    <p2> shoot kill wayn drummond two wit shoot re...
1070    mari e <p2> other appli patent upon method inc...
1035    follow censu democrat control <p1> legislatur ...
1513    <p1> tanner <p1> indict charg conspiraci defra...
Name: fact_processing, Length: 1629, dtype: object

In [ ]:
X_train["승률범주형분류"]

1       3
3       3
4       3
5       3
7       3
       ..
1617    3
1620    3
1621    3
1624    4
1627    3
Name: 승률범주형분류, Length: 1068, dtype: int64

In [ ]:
X_train_text, X_val_text, X_train_numeric, X_val_numeric, X_train_categorical, X_val_categorical, y_train, y_val = train_test_split(
    train_df["fact_processing"], X_train["승률범주형분류"], X_train[['first_party_PERSON', 'first_party_others', 'second_party_GPE',
       'second_party_ORG', 'second_party_PERSON', 'second_party_others','issued_area_civil', 'issued_area_criminal', 'issued_area_others']], X_train["first_party_winner"],
    test_size=0.25, random_state=42
)

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)


In [ ]:
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
history = model.fit(
    [X_train_text, X_train_numeric, X_train_categorical], y_train, batch_size=32,
    validation_data=([X_val_text, X_val_numeric, X_val_categorical], y_val),callbacks=[es, mc],
    epochs=30
)

Epoch 1/30
58/59 [============================>.] - ETA: 0s - loss: 0.6188 - accuracy: 0.6686
Epoch 1: val_accuracy did not improve from 0.68548
59/59 [==============================] - 30s 512ms/step - loss: 0.6190 - accuracy: 0.6685 - val_loss: 0.6141 - val_accuracy: 0.6823
Epoch 2/30
58/59 [============================>.] - ETA: 0s - loss: 0.6156 - accuracy: 0.6789
Epoch 2: val_accuracy improved from 0.68548 to 0.68710, saving model to best_model.h5
59/59 [==============================] - 35s 604ms/step - loss: 0.6158 - accuracy: 0.6787 - val_loss: 0.6107 - val_accuracy: 0.6871
Epoch 3/30
58/59 [============================>.] - ETA: 0s - loss: 0.6196 - accuracy: 0.6670
Epoch 3: val_accuracy did not improve from 0.68710
59/59 [==============================] - 28s 470ms/step - loss: 0.6200 - accuracy: 0.6668 - val_loss: 0.6182 - val_accuracy: 0.6823
Epoch 4/30
58/59 [============================>.] - ETA: 0s - loss: 0.6190 - accuracy: 0.6665
Epoch 4: val_accuracy did not improve fr

In [ ]:
test_df

,Unnamed: 0,ID,first_party,second_party,fact,issued_area,sen_len,word_len,winning_percent,승률범주형분류,fact_processing
0,0,TEST_0000,others,GPE,The 1984 Bail Reform Act allowed the federal c...,others,2,55,0.5,3,bail reform act allow feder court detain arres...
1,1,TEST_0001,PERSON,ORG,Lexecon <p2> was a defendant in a class action...,criminal,7,209,0.5,3,lexecon <p2> defend class action lawsuit usc s...
2,2,TEST_0002,ORG,ORG,"In 2002 and 2003, <p2> Stations broadcast the ...",criminal,7,181,0.5,3,<p2> station broadcast billboard music award a...
3,3,TEST_0003,PERSON,GPE,During his trial for armed robbery of a federa...,others,6,99,0.5,3,trial arm robberi feder insur save loan associ...
4,4,TEST_0004,PERSON,others,"In 1993, a magistrate judge issued a warrant a...",others,6,154,0.5,3,magistr judg issu warrant author search paul e...
...,...,...,...,...,...,...,...,...,...,...,...
1235,1235,TEST_1235,ORG,ORG,According to Executive Order No. 12807 signed ...,criminal,5,156,0.5,3,accord execut order sign presid georg h w bush...
1236,1236,TEST_1236,others,ORG,Section 109(a) of the Clean Air Act (CAA) requ...,criminal,7,221,0.5,3,section clean air act caa requir environment p...
1237,1237,TEST_1237,PERSON,PERSON,<p1> created a plan for utilizing $2.6 million...,others,12,236,0.5,3,<p1> creat plan util million fund offic hous e...
1238,1238,TEST_1238,ORG,PERSON,"In 1972, the North Carolina Board of Agricultu...",others,3,84,0.5,3,north carolina board agricultur adopt regul re...


In [ ]:
test_df["fact_processing"], X_train["승률범주형분류"], X_train[['first_party_PERSON', 'first_party_others', 'second_party_GPE',
       'second_party_ORG', 'second_party_PERSON', 'second_party_others','issued_area_civil', 'issued_area_criminal', 'issued_area_others']]

In [ ]:
X_test_text = test_df["fact_processing"]
X_test_numeric = X_train["승률범주형분류"]
X_test_categorical = X_train[['first_party_PERSON', 'first_party_others', 'second_party_GPE',
       'second_party_ORG', 'second_party_PERSON', 'second_party_others','issued_area_civil', 'issued_area_criminal', 'issued_area_others']]

In [ ]:
X_test_categorical

,first_party_PERSON,first_party_others,second_party_GPE,second_party_ORG,second_party_PERSON,second_party_others,issued_area_civil,issued_area_criminal,issued_area_others
0,1,0,0,0,1,0,0,0,1
1,1,0,0,0,1,0,0,1,0
2,1,0,1,0,0,0,0,1,0
3,0,1,0,0,0,1,0,0,1
4,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
2473,0,0,0,1,0,0,0,0,1
2474,1,0,0,1,0,0,0,0,1
2475,0,0,1,0,0,0,0,1,0
2476,0,0,0,0,0,1,0,0,1


In [ ]:
import tensorflow_hub as hub

# 모델 로드 시 'custom_objects'에 TensorFlow Hub의 'KerasLayer' 전달
best_model = load_model('best_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# 이후 작업을 위해 모델 사용


In [ ]:
X_test_text = test_df["fact_processing"]
X_test_numeric = X_test["승률범주형분류"]
X_test_categorical = X_test[['first_party_PERSON', 'first_party_others', 'second_party_GPE',
       'second_party_ORG', 'second_party_PERSON', 'second_party_others','issued_area_civil', 'issued_area_criminal', 'issued_area_others']]

In [ ]:
test_df

,Unnamed: 0,ID,first_party,second_party,fact,issued_area,sen_len,word_len,winning_percent,승률범주형분류,fact_processing
0,0,TEST_0000,others,GPE,The 1984 Bail Reform Act allowed the federal c...,others,2,55,0.5,3,bail reform act allow feder court detain arres...
1,1,TEST_0001,PERSON,ORG,Lexecon <p2> was a defendant in a class action...,criminal,7,209,0.5,3,lexecon <p2> defend class action lawsuit usc s...
2,2,TEST_0002,ORG,ORG,"In 2002 and 2003, <p2> Stations broadcast the ...",criminal,7,181,0.5,3,<p2> station broadcast billboard music award a...
3,3,TEST_0003,PERSON,GPE,During his trial for armed robbery of a federa...,others,6,99,0.5,3,trial arm robberi feder insur save loan associ...
4,4,TEST_0004,PERSON,others,"In 1993, a magistrate judge issued a warrant a...",others,6,154,0.5,3,magistr judg issu warrant author search paul e...
...,...,...,...,...,...,...,...,...,...,...,...
1235,1235,TEST_1235,ORG,ORG,According to Executive Order No. 12807 signed ...,criminal,5,156,0.5,3,accord execut order sign presid georg h w bush...
1236,1236,TEST_1236,others,ORG,Section 109(a) of the Clean Air Act (CAA) requ...,criminal,7,221,0.5,3,section clean air act caa requir environment p...
1237,1237,TEST_1237,PERSON,PERSON,<p1> created a plan for utilizing $2.6 million...,others,12,236,0.5,3,<p1> creat plan util million fund offic hous e...
1238,1238,TEST_1238,ORG,PERSON,"In 1972, the North Carolina Board of Agricultu...",others,3,84,0.5,3,north carolina board agricultur adopt regul re...


In [ ]:
predictions = best_model.predict([X_test_text, X_test_numeric, X_test_categorical])


39/39 [==============================] - 13s 334ms/step


In [ ]:
threshold = 0.5
binary_predictions = [1 if pred >= threshold else 0 for pred in predictions]

# 이진 분류 결과 출력
pd.DataFrame(binary_predictions).value_counts()

1    1231
0       9
dtype: int64

In [ ]:
submit = pd.read_csv("/content/drive/MyDrive/DACON_JudgmentPrediction/data/sample_submission.csv")

In [ ]:
submit["first_party_winner"] = binary_predictions


In [ ]:
pd.DataFrame(binary_predictions).value_counts()

1    971
0    269
dtype: int64

In [ ]:
submit.to_csv("bert_submit.csv", index=False)
